In [1]:
import glob
import uproot
import numpy as np
import os, subprocess
import boost_histogram as bh
from tqdm import tqdm

train_folder = "/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run_fixedLumis/run1"
old_direc = f"combine_grid_space_2.5"

lumi_scale = 0.1




In [2]:
# First read in all of the datacard files
datacard_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/*datacard.txt")


In [11]:
datacard_files[:2]

['/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH90.0_mA120.0/MuMu_datacard.txt',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH90.0_mA120.0/EE_datacard.txt']

In [5]:
# lumi_scales = np.arange(0.1, 0.2, 0.1)
lumi_scales = [0.01, 0.05, 0.2, 0.5]
for lumi_scale in lumi_scales:
    print(f"Doing lumi scale {lumi_scale}")

    os.makedirs(f"{train_folder}/{old_direc}_scaled{lumi_scale}", exist_ok=True)
    # First copy the masses_scan over
    cmd = f"cp {train_folder}/{old_direc}/mass_scan.txt {train_folder}/{old_direc}_scaled{lumi_scale}/mass_scan.txt"
    status, out = subprocess.getstatusoutput(cmd)
    print(out)

    for f in datacard_files:
        _, mass, file = f.split(f"{old_direc}/")[1].split("/")
        
        # Now move these into the new combine directory

        new_direc = f"{train_folder}/{old_direc}_scaled{lumi_scale}/combine/{mass}"
        os.makedirs(new_direc, exist_ok=True)

        # Now move the file into there
        cmd = f"cp {f} {new_direc}/{file}"
        status, out = subprocess.getstatusoutput(cmd)



    # First read in all of the datacard files
    root_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/backgrounds.root")
    root_files[:3]

    for file_name_full in tqdm(root_files):
        file = uproot.open(file_name_full)

        _, mass, filename = file_name_full.split(f"{old_direc}/")[1].split("/")

        file.keys()

        new_dict = {}
        bins = np.linspace(0.9, 1, 25)

        for key in file.keys():
            bhist = file[key].to_boost()

            hist = bhist.counts()
            sumw2 = bhist.variances()

            hist = hist * lumi_scale
            sumw2 = sumw2 * lumi_scale**2


            root_hist = bh.Histogram(bh.axis.Variable(bins), 
                                    storage=bh.storage.Weight())
            root_hist[...] = np.stack([hist, sumw2], axis=-1)


            new_dict[key[:-2]] = root_hist



        
        # Now write this to the new directory
        with uproot.recreate(f"{train_folder}/{old_direc}_scaled{lumi_scale}/combine/{mass}/{filename}") as f:
            for key, hist in new_dict.items():
                f[key] = hist
    

    # First read in all of the datacard files
    root_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/*hists.root")
    root_files[:3]

    for file_name_full in tqdm(root_files):
        file = uproot.open(file_name_full)

        _, mass, filename = file_name_full.split(f"{old_direc}/")[1].split("/")

        file.keys()

        new_dict = {}
        bins = np.linspace(0.9, 1, 25)

        for key in file.keys():
            bhist = file[key].to_boost()

            hist = bhist.counts()
            sumw2 = bhist.variances()

            hist = hist * lumi_scale
            sumw2 = sumw2 * lumi_scale**2


            root_hist = bh.Histogram(bh.axis.Variable(bins), 
                                    storage=bh.storage.Weight())
            root_hist[...] = np.stack([hist, sumw2], axis=-1)


            new_dict[key[:-2]] = root_hist



        
        # Now write this to the new directory
        with uproot.recreate(f"{train_folder}/{old_direc}_scaled{lumi_scale}/combine/{mass}/{filename}") as f:
            for key, hist in new_dict.items():
                f[key] = hist

Doing lumi scale 0.01



100%|██████████| 1368/1368 [00:38<00:00, 35.93it/s]


Doing lumi scale 0.05



100%|██████████| 1368/1368 [00:21<00:00, 63.05it/s]


Doing lumi scale 0.2



100%|██████████| 1368/1368 [00:21<00:00, 62.87it/s]


Doing lumi scale 0.5



100%|██████████| 1368/1368 [00:19<00:00, 70.39it/s]


In [59]:
# First read in all of the datacard files
root_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/backgrounds.root")
root_files[:3]

for file_name_full in tqdm(root_files):
    file = uproot.open(file_name_full)

    _, mass, filename = file_name_full.split(f"{old_direc}/")[1].split("/")

    file.keys()

    new_dict = {}
    bins = np.linspace(0.9, 1, 25)

    for key in file.keys():
        bhist = file[key].to_boost()

        hist = bhist.counts()
        sumw2 = bhist.variances()

        hist = hist * lumi_scale
        sumw2 = sumw2 * lumi_scale**2


        root_hist = bh.Histogram(bh.axis.Variable(bins), 
                                storage=bh.storage.Weight())
        root_hist[...] = np.stack([hist, sumw2], axis=-1)


        new_dict[key] = root_hist



    
    # Now write this to the new directory
    with uproot.recreate(f"{train_folder}/combine_grid_space_2.5_scaled{lumi_scale}/combine/{mass}/{filename}") as f:
        for key, hist in new_dict.items():
            f[key] = hist

100%|██████████| 684/684 [00:46<00:00, 14.82it/s]


In [62]:
# First read in all of the datacard files
root_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/*hists.root")
root_files[:3]

for file_name_full in tqdm(root_files):
    file = uproot.open(file_name_full)

    _, mass, filename = file_name_full.split(f"{old_direc}/")[1].split("/")

    file.keys()

    new_dict = {}
    bins = np.linspace(0.9, 1, 25)

    for key in file.keys():
        bhist = file[key].to_boost()

        hist = bhist.counts()
        sumw2 = bhist.variances()

        hist = hist * lumi_scale
        sumw2 = sumw2 * lumi_scale**2


        root_hist = bh.Histogram(bh.axis.Variable(bins), 
                                storage=bh.storage.Weight())
        root_hist[...] = np.stack([hist, sumw2], axis=-1)


        new_dict[key] = root_hist



    
    # Now write this to the new directory
    with uproot.recreate(f"{train_folder}/combine_grid_space_2.5_scaled{lumi_scale}/combine/{mass}/{filename}") as f:
        for key, hist in new_dict.items():
            f[key] = hist

100%|██████████| 1368/1368 [00:29<00:00, 46.55it/s]


In [60]:
root_files = glob.glob(f"{train_folder}/{old_direc}/combine*/*/*hists.root")

In [61]:
root_files

['/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH90.0_mA120.0/mH90.0_mA120.0_signal_Electron_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH90.0_mA120.0/mH90.0_mA120.0_signal_Muon_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH102.5_mA132.5/mH102.5_mA132.5_signal_Muon_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH102.5_mA132.5/mH102.5_mA132.5_signal_Electron_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH92.5_mA110.0/mH92.5_mA110.0_signal_Muon_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH92.5_mA110.0/mH92.5_mA110.0_signal_Electron_hists.root',
 '/vols/cms/emc21/FCC/FCC-Study/runs/e240_full_run/run25/combine_grid_space_2.5/combine/mH72.5_mA102.5/mH72.5_mA102.5_signal_E

In [56]:
file_name_full = root_files[0]
file = uproot.open(file_name_full)

_, mass, filename = file_name_full.split(f"{old_direc}/")[1].split("/")

file.keys()

new_dict = {}
bins = np.linspace(0.9, 1, 25)

for key in file.keys():
    bhist = file[key].to_boost()

    hist = bhist.counts()
    sumw2 = bhist.variances()

    hist = hist * lumi_scale
    sumw2 = sumw2 * lumi_scale**2


    root_hist = bh.Histogram(bh.axis.Variable(bins), 
                            storage=bh.storage.Weight())
    root_hist[...] = np.stack([hist, sumw2], axis=-1)


    new_dict[key] = root_hist



 
# Now write this to the new directory
with uproot.recreate(f"{train_folder}/combine_grid_space_2.5_scaled{lumi_scale}/combine/{mass}/{filename}") as f:
    for key, hist in new_dict.items():
        f[key] = hist


In [43]:
dir(file['ZZ_Electron;1'].to_boost())

['__add__',
 '__annotations__',
 '__array__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__itruediv__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '_axis',
 '_clone',
 '_compute_commonindex',
 '_compute_inplace_op',
 '_compute_uhi_index',
 '_export_bh_',
 '_family',
 '_from_histogram_cpp',
 '_from_histogram_object',
 '_generate_axes_',
 '_hist',
 '_import_bh_',
 '_new_hist',
 '_reduce',
 '_storage_type',
 '_types',
 '_variance_known',
 'axes',
 'copy',
 'counts',
 'empty',
 'fill',
 '

In [46]:
file['ZZ_Electron;1'].to_boost().to_numpy()

(array([  2.6171875,   2.609375 ,   1.7421875,   2.6171875,   7.8359375,
         11.3203125,   5.234375 ,   4.3515625,   3.484375 ,   2.6171875,
          6.09375  ,  10.453125 ,  12.1953125,  14.8046875,  14.8125   ,
         16.546875 ,  17.421875 ,  26.140625 ,  26.125    ,  54.015625 ,
        113.234375 , 235.203125 , 467.7734375, 342.3359375]),
 array([0.9       , 0.90416667, 0.90833333, 0.9125    , 0.91666667,
        0.92083333, 0.925     , 0.92916667, 0.93333333, 0.9375    ,
        0.94166667, 0.94583333, 0.95      , 0.95416667, 0.95833333,
        0.9625    , 0.96666667, 0.97083333, 0.975     , 0.97916667,
        0.98333333, 0.9875    , 0.99166667, 0.99583333, 1.        ]))

In [45]:
file['ZZ_Electron;1'].to_boost().counts()

array([  2.6171875,   2.609375 ,   1.7421875,   2.6171875,   7.8359375,
        11.3203125,   5.234375 ,   4.3515625,   3.484375 ,   2.6171875,
         6.09375  ,  10.453125 ,  12.1953125,  14.8046875,  14.8125   ,
        16.546875 ,  17.421875 ,  26.140625 ,  26.125    ,  54.015625 ,
       113.234375 , 235.203125 , 467.7734375, 342.3359375])

In [47]:
file['ZZ_Electron;1'].to_boost().variances()

array([ 1.50779226,  1.50779226,  1.23110723,  1.50779226,  2.6115728 ,
        3.13871988,  2.13234026,  1.94655144,  1.74104853,  1.50779226,
        2.30319072,  3.01558452,  3.25720356,  3.5892635 ,  3.5892635 ,
        3.79452731,  3.89310288,  4.76805778,  4.76805778,  6.85451494,
        9.92550377, 14.30417334, 20.17288558, 17.25747121])

In [38]:
dir(file['ZZ_Electron'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bases',
 '_concrete',
 '_cursor',
 '_deeply_writable',
 '_file',
 '_instance_version',
 '_is_memberwise',
 '_members',
 '_num_bytes',
 '_parent',
 '_serialize',
 '_to_boost_histogram_',
 '_to_writable',
 '_to_writable_postprocess',
 '_values',
 '_values_variances',
 'all_members',
 'awkward_form',
 'axes',
 'axis',
 'base',
 'base_names_versions',
 'bases',
 'behaviors',
 'check_numbytes',
 'class_code',
 'class_flags',
 'class_rawstreamers',
 'class_streamer',
 'class_version',
 'classname',
 'close',
 'closed',
 'concrete',
 'counts',
 'cursor',
 'empty',
 '